In [1]:
!python3 -m pip install tensorflow-gpu==1.15 tqdm

     |████████████████████████████████| 411.5MB 37kB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 3.8MB 37.9MB/s 
     |████████████████████████████████| 512kB 39.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=ebaa8e2e547c60028d3efcc99d597141d6a5cd37214fe4db8f9bbdd22b84cea1
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing i

In [2]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
sess = tf.InteractiveSession()

In [3]:
class N3D():
  def __init__(self, n):
    self.filters = []
    self.biases = []
    for i in range(n):
      self.filters.append(tf.random.normal([3, 3, 3, 32, 32], mean=0.0, stddev=1.0, dtype=tf.dtypes.float32, name="f%d"%i))
      self.biases.append(tf.random.normal([1, 1, 1, 1, 32], mean=0.0, stddev=1.0, dtype=tf.dtypes.float32, name="b%d"%i))
  
  def gen_graph(self, x):
    # x = (N, D, H, W, 32)
    for f, b in zip(self.filters, self.biases):
      x = tf.nn.conv3d(x, filter=f, strides=[1, 1, 1, 1, 1], padding="SAME", data_format='NDHWC', dilations=[1, 1, 1, 1, 1])
      x = x + b
    
    return x

class N2D():
  def __init__(self, n):
    self.filters1 = []
    self.filters2 = []
    self.filters3 = []
    self.biases = []
    for i in range(n):
      self.filters1.append(tf.random.normal([3, 1, 32, 32], mean=0.0, stddev=1.0, dtype=tf.dtypes.float32, name="f1_%d"%i)) # N*D, H, W, C
      self.filters2.append(tf.random.normal([1, 3, 32, 32], mean=0.0, stddev=1.0, dtype=tf.dtypes.float32, name="f2_%d"%i)) # N*D, H, W, C
      self.filters3.append(tf.random.normal([3, 1, 32, 32], mean=0.0, stddev=1.0, dtype=tf.dtypes.float32, name="f3_%d"%i)) # N, D, H*W, C
      self.biases.append(tf.random.normal([1, 1, 1, 32], mean=0.0, stddev=1.0, dtype=tf.dtypes.float32, name="b_%d"%i)) # N, D, H*W, C
  
  def gen_graph(self, x, n, d, h, w, c): # n is batch size
    # x = (N*D, H, W, 3)
    for f1, f2, f3, b in zip(self.filters1, self.filters2, self.filters3, self.biases):
      x = tf.nn.conv2d(x, filter=f1, strides=[1, 1, 1, 1], padding="SAME", data_format='NHWC', dilations=[1, 1, 1, 1])
      x = tf.nn.conv2d(x, filter=f2, strides=[1, 1, 1, 1], padding="SAME", data_format='NHWC', dilations=[1, 1, 1, 1])
      x = tf.reshape(x, [n, -1, h*w, c])
      x = tf.nn.conv2d(x, filter=f3, strides=[1, 1, 1, 1], padding="SAME", data_format='NHWC', dilations=[1, 1, 1, 1])
      x = x + b
      x = tf.reshape(x, [n*d, h, w, c])
    
    return x

In [4]:
n1 = N3D(10)
n2 = N2D(10)

x1 = tf.placeholder(dtype=tf.float32, shape=[2, 8, 128, 128, 32])
x2 = tf.placeholder(dtype=tf.float32, shape=[2*8, 128, 128, 32])

y1 = n1.gen_graph(x1)
y2 = n2.gen_graph(x2, 2, 8, 128, 128, 32)

In [5]:
xx = np.random.rand(2, 8, 128, 128, 32)
xx2 = np.random.rand(2*8, 128, 128, 32)

for _ in tqdm(range(1000)):
  _ = sess.run(y1, feed_dict={x1: xx})

for _ in tqdm(range(1000)):
  _ = sess.run(y2, feed_dict={x2: xx2})

100%|██████████| 1000/1000 [01:49<00:00,  9.11it/s]
